# Assignment 2
This file covers the solutions to the 3 questions on assignment 2.
1. Implement C4.5 decision tree classifier.
2. Evaluate the impact of noise on C4.5 decision tree classifier.
3. Design a feature selection algorithm and apply to MNIST dataset.

In [431]:
# import packages
import pandas
import numpy as np
import scipy as sp
import scipy.linalg as sl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from mlxtend.data import loadlocal_mnist
from anytree import Node, RenderTree

## Question 1

In [432]:
# load glass and tic tac toe datasets
glass_dataframe = pandas.read_csv('datasets/glass.data', header=None, engine='python')
tictactoe_dataframe = pandas.read_csv('datasets/tic-tac-toe.data', header=None, engine='python')

# label encoding for tictactoe targets
le = LabelEncoder()
tictactoe_dataframe[9] = le.fit_transform(tictactoe_dataframe[9])

# seperate featres from targets
glass_X = glass_dataframe.iloc[:,1:-1].values
glass_Y = glass_dataframe.iloc[:,-1].values
tictactoe_X = tictactoe_dataframe.iloc[:,:-1].values
tictactoe_Y = tictactoe_dataframe.iloc[:,-1].values

# feature names for each datset
glass_feature_names = np.array(['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'])

tictactoe_feature_names = np.array(['top-left', 'top-middle', 'top-right',
                          'middle-left', 'middle-middle', 'middle-right',
                          'bottom-left', 'bottom-middle', 'bottom-right'])

In [433]:
glass_dataframe.nunique()

0     214
1     178
2     142
3      94
4     118
5     133
6      65
7     143
8      34
9      32
10      6
dtype: int64

In [434]:
glass_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 11 columns):
0     214 non-null int64
1     214 non-null float64
2     214 non-null float64
3     214 non-null float64
4     214 non-null float64
5     214 non-null float64
6     214 non-null float64
7     214 non-null float64
8     214 non-null float64
9     214 non-null float64
10    214 non-null int64
dtypes: float64(9), int64(2)
memory usage: 18.5 KB


In [435]:
tictactoe_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
0    958 non-null object
1    958 non-null object
2    958 non-null object
3    958 non-null object
4    958 non-null object
5    958 non-null object
6    958 non-null object
7    958 non-null object
8    958 non-null object
9    958 non-null int32
dtypes: int32(1), object(9)
memory usage: 71.2+ KB


In [436]:
# information gain debugging
data = tictactoe_X[:,4]
idx_subsets = []
for feature_val in np.unique(data):
    idx_subset = np.argwhere(data == feature_val).ravel()
    idx_subsets.append(idx_subset)
subsets = [tictactoe_Y[idx] for idx in idx_subsets]

informationGain(tictactoe_Y, subsets)

0.08718670223056812

In [437]:
# best feature selection debugging
ft_idx, ft_split, ft_threshold, ft_discrete = selectBestAttribute(tictactoe_X, tictactoe_Y, tictactoe_feature_names)
lengths = [len(x) for x in ft_split]
print(tictactoe_feature_names[ft_idx])
print(lengths)
sum(lengths)

['middle-middle']
[160, 340, 458]


958

In [438]:
# implement C4.5 Decision Tree
def buildTree(x, y, features):
    numfeatures = len(features)
    numsamples = x.shape[0]
    
    # base case 1: check if attribute list empty
    if len(features) == 0:
        isLeafNode= True
        majorityClass = np.bincount(y).argmax()
        return dtNode(None, isLeafNode, majorityClass, None, None)
    
    # base case 2: check if all samples are of the same class
    elif len(set(y)) == 1:
        isLeafNode= True
        classLabel = y[0]
        return dtNode(None, isLeafNode, classLabel, None, None)
    
    # select next best feature and make recursive call to branch out
    else:
        isLeafNode = False
        majorityClass = np.bincount(y).argmax()
        feature_idx, feature_splits, threshold, isFeatureDiscrete = selectBestAttribute(x, y, features)
        
        # determine remaining features and remove best feature selected
        new_features = np.delete(features, feature_idx)
        new_x = np.delete(x, feature_idx, axis=1)
        
        # iterate over feature splits and create child nodes
        node = dtNode(threshold, isLeafNode, majorityClass, features[feature_idx], isFeatureDiscrete)
        for split in feature_splits:
            node.childNodes.append(buildTree(new_x[split], y[split], new_features))
        return node
        

# select best attribute based on gain ratio
def selectBestAttribute(x, y, features):
    best_feature_idx = None
    best_feature_splits = None
    best_feature_threshold = None
    best_feature_discrete = None
    best_gain = -np.inf
    
    for feature in features:
        feature_idx = np.argwhere(features == feature).ravel()
        data = x[:,feature_idx].ravel()
        
        # dealing with discrete features
        isFeatureDiscrete = isDiscrete(data)
        if isFeatureDiscrete:
            feature_values = np.unique(data)
            idx_subsets = []
            for feature_val in feature_values:
                idx_subset = np.argwhere(data == feature_val).ravel()
                idx_subsets.append(idx_subset)
            y_subsets = [y[idx] for idx in idx_subsets]
            
            gain = informationGain(y, y_subsets)
            if gain > best_gain:
                best_gain = gain
                best_feature_idx = feature_idx
                best_feature_splits = idx_subsets
                best_feature_threshold = feature_values
                best_feature_discrete = True
        
        # dealing with continuous features
        else:
            sortedData = np.sort(data)
            
            # iterate over all possible thresholds
            for i in range(len(sortedData)-1):
                if sortedData[i] != sortedData[i+1]:
                    pickThreshold = (sortedData[i] + sortedData[i+1]) / 2
                    leftIdx = np.argwhere(data <= pickThreshold).ravel()
                    rightIdx = np.argwhere(data > pickThreshold).ravel()
                    
                    idx_subsets = [leftIdx, rightIdx]
                    y_subsets = [y[idx] for idx in idx_subsets]
                                      
                    gain = informationGain(y, y_subsets)
                    if gain > best_gain:
                        best_gain = gain
                        best_feature_idx = feature_idx
                        best_feature_splits = idx_subsets
                        best_feature_threshold = pickThreshold
                        best_feature_discrete = False
    
    return (best_feature_idx, best_feature_splits, best_feature_threshold, best_feature_discrete)

# check if feature is discrete or continuous using threshold of 10
def isDiscrete(data):
    if len(np.unique(data)) < 10:
        return True
    else:
        return False
        
    
# gain ratio
def gainRatio():
    return informationGain() / splitInformation()

# information gain
def informationGain(y, y_subsets):
    numsamples = len(y)
    numsubsets = len(y_subsets)
    
    # calculate initialEntropy
    initialEntropy = entropy(y)
    
    # calculate afterEntropy
    afterEntropy = 0
    for i in range(numsubsets):
        subset_weight = len(y_subsets[i]) / numsamples
        afterEntropy += subset_weight * entropy(y_subsets[i])
    
    return initialEntropy - afterEntropy

# split information
def splitInformation():
    pass

# entropy calculation
def entropy(y):
    counts = np.bincount(y)
    probabilities = counts[np.nonzero(counts)] / float(len(y))
    return sp.stats.entropy(probabilities, base=2)


In [439]:
# predict the class_label of sample given the C4.5 tree
def predictClass(dt, x, feature_names):
    currNode = dt
    leafNode = dt.isLeafNode
    while not leafNode:
        idx = np.argwhere(feature_names == currNode.feature).ravel()
        
        # handling decision node that is discrete
        if currNode.isFeatureDiscrete:
            split_idx = np.argwhere(currNode.threshold == x[idx]).ravel()
            
            if len(split_idx):
                currNode = currNode.childNodes[split_idx[0]]
                leafNode = currNode.isLeafNode
                
            else:
                break
        
        # handling decision node that is continuous
        else: 
            if x[idx] <= currNode.threshold:
                currNode = currNode.childNodes[0] # go left in the decision path
            else:
                currNode = currNode.childNodes[1] # go right in the decision path

            leafNode = currNode.isLeafNode
    
    y_pred = currNode.label
    return y_pred

In [440]:
# decision tree class that uses C4.5 implementation
class DecisionTree:
    
    # if hyperparamters required, they are to be added here (ex. max_depth)
    def __init__(self, feature_names):
        self.clf = None
        self.feature_names = feature_names
    
    def fit(self, x_train, y_train):
        self.clf = buildTree(x_train, y_train, self.feature_names)
    
    def predict(self, x_test):
        y_pred = []
        for x in x_test:
            y_pred.append(predictClass(self.clf, x, self.feature_names))
            
        return y_pred

    
# decision tree node that defines threshold, isLeafNode, class_label and childNodes list
class dtNode:
    
    def __init__(self, threshold, isLeafNode, label, feature, isFeatureDiscrete):
        self.threshold = threshold # threshold for the decision path, None if isLeafNode
        self.isLeafNode = isLeafNode  # if current node is a leaf (terminal) node
        self.label = label  # the class label, if not isLeafNode it is the majority class label
        self.feature = feature # if not isLeafNode it is the feature to split on, otherwise None
        self.isFeatureDiscrete = isFeatureDiscrete # check feature discrete or continuous, None if isLeafNode
        self.childNodes = []
        

In [441]:
# perform CV with given number of iterations and folds
def crossvalidation(clf, X, Y, iters=10, k=10):
    numsamples = X.shape[0]

    accuracy_scores = []
    for i in range(iters):

        # single iteration of k-fold CV
        idxs = np.arange(numsamples)
        folds = np.random.choice(idxs, size=(k, int(numsamples/k)), replace=False)

        # obtain accuracy scores for k-fold CV
        kfold_accuracy_scores = []
        for j in range(k):
            test_idx = folds[j]
            train_idx = np.delete(folds, j, axis=0).ravel()

            clf.fit(X[train_idx], Y[train_idx])
            y_pred = clf.predict(X[test_idx])

            accuracy = accuracy_score(Y[test_idx], y_pred)
            kfold_accuracy_scores.append(accuracy)

        accuracy_scores.append(np.mean(kfold_accuracy_scores))
    
    return accuracy_scores


In [442]:
# build classifier on glass dataset
glass_clf = DecisionTree(glass_feature_names)

In [443]:
# do 10 times 10 CV on tic glass dataset
accuracy_scores = crossvalidation(glass_clf, glass_X, glass_Y, iters=10, k=10)

print("Glass Dataset")
print("10 times 10 fold CV: (0.95 CI)")
print("-" * 30)
print(f"Accuracy: {np.mean(accuracy_scores):.3f} +/- {2*np.std(accuracy_scores, ddof=0):.3f}")

Glass Dataset
10 times 10 fold CV: (0.95 CI)
------------------------------
Accuracy: 0.654 +/- 0.028


In [444]:
# build classifier on tic tac toe dataset
tictactoe_clf = DecisionTree(tictactoe_feature_names)

In [445]:
# do 10 times 10 CV on tic tac toe dataset
accuracy_scores = crossvalidation(tictactoe_clf, tictactoe_X, tictactoe_Y, iters=10, k=10)

print("Tic Tac Toe Dataset")
print("10 times 10 fold CV: (0.95 CI)")
print("-" * 30)
print(f"Accuracy: {np.mean(accuracy_scores):.3f} +/- {2*np.std(accuracy_scores, ddof=0):.3f}")

Tic Tac Toe Dataset
10 times 10 fold CV: (0.95 CI)
------------------------------
Accuracy: 0.857 +/- 0.021


## Question 2

## Question 3

In [446]:
# load mnist dataset
X_train, Y_train = loadlocal_mnist(
        images_path='datasets/train-images.idx3-ubyte', 
        labels_path='datasets/train-labels.idx1-ubyte')